In [16]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np

In [17]:

from tensorflow import keras
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Activation
from tensorflow.keras.layers import BatchNormalization

import numpy as np

In [18]:
pip install tensorflow-addons==0.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
# Get Data
import tflearn.datasets.oxflower17 as oxflower17
x,y =oxflower17.load_data(one_hot=True)


In [21]:
x

array([[[[0.01176471, 0.01176471, 0.00392157],
         [0.01176471, 0.01176471, 0.00392157],
         [0.01176471, 0.01176471, 0.00392157],
         ...,
         [0.08627451, 0.07843138, 0.05882353],
         [0.11764706, 0.08627451, 0.06666667],
         [0.12156863, 0.08627451, 0.06666667]],

        [[0.01176471, 0.01176471, 0.00392157],
         [0.01176471, 0.01176471, 0.00392157],
         [0.01176471, 0.01176471, 0.00392157],
         ...,
         [0.08627451, 0.08235294, 0.0627451 ],
         [0.11764706, 0.08627451, 0.06666667],
         [0.12156863, 0.08627451, 0.06666667]],

        [[0.01176471, 0.01176471, 0.00392157],
         [0.01176471, 0.01176471, 0.00392157],
         [0.01176471, 0.01176471, 0.00392157],
         ...,
         [0.09411765, 0.08235294, 0.0627451 ],
         [0.1254902 , 0.09411765, 0.07450981],
         [0.12941177, 0.09411765, 0.07450981]],

        ...,

        [[0.16470589, 0.11764706, 0.08627451],
         [0.17254902, 0.1254902 , 0.10588235]

In [22]:
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [23]:
x.shape,y.shape


((1360, 224, 224, 3), (1360, 17))

In [24]:
# Create a sequential model
model = Sequential()

# 1st Convolutional Layer ,maxpool,batch normalization
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))
model.add(BatchNormalization()) ## to mitigate the unstable gradients within NN

# 2nd Convolutional Layer,maxpool,batch normalization
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))
model.add(BatchNormalization())

# 3rd Convolutional Layer,batch normalization
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid',activation='relu'))
model.add(BatchNormalization())

# 4th Convolutional Layer,batch normalization
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid',activation='relu'))
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid',))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())


# Passing it to a dense layer
model.add(Flatten())

# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 batch_normalization_7 (Batc  (None, 26, 26, 96)       384       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 12, 12, 256)      0         
 2D)                                                             
                                                      

In [26]:
model.compile(loss=keras.metrics.categorical_crossentropy ,optimizer=keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy'])

In [28]:
# Train
model.fit(x, y, batch_size=128, epochs=20, verbose=1,validation_split=0.2, shuffle=True)

Train on 1088 samples, validate on 272 samples
Epoch 1/20
1088/1088 [==============================] - 5s 4ms/sample - loss: 3.5511 - acc: 0.3189 - val_loss: 48.9401 - val_acc: 0.1471
Epoch 2/20
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.8424 - acc: 0.3539 - val_loss: 29.0685 - val_acc: 0.1691
Epoch 3/20
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.5680 - acc: 0.3998 - val_loss: 23.6043 - val_acc: 0.1765
Epoch 4/20
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.2654 - acc: 0.4256 - val_loss: 12.5090 - val_acc: 0.1949
Epoch 5/20
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.3128 - acc: 0.4697 - val_loss: 10.7507 - val_acc: 0.2537
Epoch 6/20
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.3963 - acc: 0.4430 - val_loss: 7.2188 - val_acc: 0.2941
Epoch 7/20
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.0146 - acc: 0.4926 - val_loss: 7.5979 - val_acc: 0.2

In [30]:
score = model.evaluate(x_test,y_test)
score=model.evaluate(x_test,y_test)
print('Test Loss:',score[0])
print('Test Accuracy:',score[1])

NameError: ignored